# RNN y Character-level Neural LM (LOTR)
Basado en: https://chunml.github.io/ChunML.github.io/project/Creating-Text-Generator-Using-Recurrent-Neural-Network/
<br><br>El cuaderno de trabajo presenta una aplicación directa de un **Character-Level Neural Language Model** para generar automáticamente texto en base a un archivo/documento textual de entrada. En este caso, se usaron los libros de LOTR (The Lord of the Rings)

<br>**Se recomienda generar un notebook nuevo, colocando sólo las instrucciones útiles para su prueba de generación de textos. Modifique y ordene lo que considere conveniente para una mayor legibilidad y comprensión de su prueba (para la revisión con los JPs en el Lab. 10)**

### Preparación (_utils_)

In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import time
import csv
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers.wrappers import TimeDistributed
import pickle

ModuleNotFoundError: No module named 'keras'

**Definición de argumentos**

In [7]:
#Archivo de texto 
DATA_DIR = "./lotr.txt" 
#Modificar BATCH_SIZE o HIDDEN_DIM en caso tengan problemas de memoria
BATCH_SIZE = 50 
HIDDEN_DIM = 250 #500
#Parametro para longitud de secuencia a analizar
SEQ_LENGTH = 50 
#Parametro para cargar un pesos previamente entrenados (checkpoint)
WEIGHTS = '' 

#Parametro para indicar cuantos caracteres generar en cada prueba
GENERATE_LENGTH = 500 
#Parametros para la red neuronal
LAYER_NUM = 2 
NB_EPOCH = 20

**Función A:
<br>(1) Carga de un archivo de texto, (2) Construcción de estructuras de entrada y salida de la red**

In [3]:
# method for preparing the training data
def load_data(data_dir, seq_length):
    #Carga del archivo
    data = open(data_dir, 'r').read()
    #Caracteres unicos
    chars = list(set(data))
    VOCAB_SIZE = len(chars)

    print('Data length: {} characters'.format(len(data)))
    print('Vocabulary size: {} characters'.format(VOCAB_SIZE))
    print(chars)
    
    #Indexacion de los caracteres
    ix_to_char = {ix:char for ix, char in enumerate(chars)}
    char_to_ix = {char:ix for ix, char in enumerate(chars)}
    
    #Estructuras de entrada y salida
    NUMBER_OF_SEQ = int(len(data)/seq_length)
    print('Number of sequences: {}'.format(NUMBER_OF_SEQ))
    X = np.zeros((NUMBER_OF_SEQ, seq_length, VOCAB_SIZE))
    y = np.zeros((NUMBER_OF_SEQ, seq_length, VOCAB_SIZE))
    
    for i in range(0, NUMBER_OF_SEQ):
        #LLenado de la estructura de entrada X
        X_sequence = data[i*seq_length:(i+1)*seq_length]
        X_sequence_ix = [char_to_ix[value] for value in X_sequence]
        #one-hot-vector (input)
        input_sequence = np.zeros((seq_length, VOCAB_SIZE))  
        #uso del diccionario para completar el one-hot-vector
        for j in range(seq_length):
            input_sequence[j][X_sequence_ix[j]] = 1.
            X[i] = input_sequence
            
        #Llenado de la estructura de salida y
        y_sequence = data[i*seq_length+1:(i+1)*seq_length+1]
        y_sequence_ix = [char_to_ix[value] for value in y_sequence]
        #one-hot-vector (output)
        target_sequence = np.zeros((seq_length, VOCAB_SIZE))
        #uso del diccionario para completar el one-hot-vector
        for j in range(seq_length):
            target_sequence[j][y_sequence_ix[j]] = 1.
            y[i] = target_sequence
            
    return X, y, VOCAB_SIZE, ix_to_char

**Función B:
<br>Generación de textos**

In [4]:
# method for generating text
def generate_text(model, length, vocab_size, ix_to_char):
    # starting with random character
    ix = [np.random.randint(vocab_size)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, vocab_size))
    for i in range(length):
        # appending the last predicted character to sequence
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)

## Entrenamiento y Prueba

**Uso de la Función A: carga de los datos**

In [6]:
# Creating training data
X, y, VOCAB_SIZE, ix_to_char = load_data(DATA_DIR, SEQ_LENGTH)

Data length: 822197 characters
Vocabulary size: 84 characters
['Y', ',', '\n', '3', 'O', '\\', 'x', 't', 'F', 'l', 'D', '_', 'V', 'd', 'E', 'j', 'L', 'T', '1', 'v', '"', 'k', 'W', 'i', 'q', 'Q', 'C', 'B', 'A', 'R', 'g', '’', 'X', 'I', 'N', 'S', '.', '®', ')', '8', 'J', '7', 'm', '0', 'r', 'e', "'", 'u', '!', '4', ' ', '5', 'Z', 'c', ';', 'H', 'G', 'U', 'w', '“', 'p', '?', 'P', 's', '*', '—', '2', '/', 'h', '(', 'y', 'o', '9', ':', 'K', 'f', 'b', 'M', 'z', 'n', '6', '-', 'a', '^']
Number of sequences: 16443


**Es importante guardar el diccionario `ix_to_char` en un archivo binario. Este debe ser cargado cada vez que se quiera retomar el entrenamiento o generar texto a partir de un checkpoint, debido a que el orden de los caracteres en el diccionario podría modificarse (no es un orden fijo)**
<br>**NO MODIFICAR ESTE PICKLE AL REINICIAR EL NOTEBOOK PARA PROBAR CHECKPOINTS**

In [25]:
#No modificar el pickle al reiniciar el cuaderno de trabajo para probar checkpoints previos
with open('ix_to_char.pickle', 'wb') as handle:
    pickle.dump(ix_to_char, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [26]:
print(ix_to_char)

{0: '’', 1: '"', 2: 'y', 3: ')', 4: '0', 5: 'z', 6: 'G', 7: 'Q', 8: '3', 9: 'N', 10: '-', 11: '*', 12: ':', 13: ' ', 14: 'F', 15: 'c', 16: '1', 17: '6', 18: 'x', 19: 'R', 20: 'U', 21: 'n', 22: '5', 23: '(', 24: 'g', 25: 'm', 26: 'l', 27: 'v', 28: 'Z', 29: 'h', 30: 'A', 31: 'e', 32: 'W', 33: 'H', 34: 'i', 35: ';', 36: 'D', 37: 'P', 38: 'b', 39: 'o', 40: "'", 41: 's', 42: 'q', 43: 'Y', 44: '7', 45: '2', 46: 't', 47: 'O', 48: '“', 49: 'p', 50: '®', 51: 'J', 52: 'C', 53: 'r', 54: '9', 55: '^', 56: ',', 57: 'V', 58: 'd', 59: '_', 60: '.', 61: 'u', 62: '\n', 63: '8', 64: 'S', 65: 'a', 66: 'T', 67: 'B', 68: 'K', 69: '?', 70: 'E', 71: 'w', 72: '!', 73: 'M', 74: 'k', 75: 'I', 76: 'j', 77: 'f', 78: '/', 79: '\\', 80: '—', 81: 'X', 82: '4', 83: 'L'}


In [27]:
print(X.shape, y.shape, VOCAB_SIZE)

(16443, 50, 84) (16443, 50, 84) 84


**Creación de la RNN (LSTM)**

In [9]:
# Creating and compiling the Network
model = Sequential()

#Añadiendo las capas LSTM
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LAYER_NUM - 1):
    model.add(LSTM(HIDDEN_DIM, return_sequences=True))
#Añadiendo la operacion de salida
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))

#"Compilando" = instanciando la RNN con su función de pérdida y optimización
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

**Prueba inicial de creación de 500 caracteres**

In [29]:
# Generate some sample before training to know how bad it is!
generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, ix_to_char)

CAzzz,,,!!!!JJ))ZZZZpZpZZprrrr;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;

'CAzzz,,,!!!!JJ))ZZZZpZpZZprrrr;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;'

**Se cargan los pesos (y el diccionario de los one-hot-vectors) en caso haya habido un entrenamiento previo**
<br>WEIGHTS debe tener el valor del nombre del archivo de "checkpoint" guardado. Por ejemplo:
<br>```WEIGHTS = "checkpoint_layer_2_hidden_250_epoch_60.hdf5"```

In [29]:
#Se cargan los pesos de un entrenamiento previo (si se desea restaurar una ejecucion)
#Se calcula el numero de epocas en base al nombre del archivo
#Se carga el diccionario de caracteres (one-hot-vectors) para la generacion
if not WEIGHTS == '':
    model.load_weights(WEIGHTS)
    nb_epoch = int(WEIGHTS[WEIGHTS.rfind('_') + 1:WEIGHTS.find('.')])
    with open('ix_to_char.pickle', 'rb') as handle:
        ix_to_char = pickle.load(handle)
else:
    #Si se va a empezar de 0:
    nb_epoch = 0

**ENTRENAMIENTO**

In [ ]:
# Training if there is no trained weights specified

#Esta es la iteración importante
#Pueden cambiar la condición para que termine en un determinado numero de epochs.
while True:
    print('\n\nEpoch: {}\n'.format(nb_epoch))
    #Ajuste del modelo, y entrenamiento de 1 epoca
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, epochs=1)
    nb_epoch += 1
    #Generacion de un texto al final de la epoca
    generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, ix_to_char)
    #Pueden modificar esto para tener más checkpoints
    if nb_epoch % 10 == 0:
        model.save_weights('checkpoint_layer_{}_hidden_{}_epoch_{}.hdf5'.format(LAYER_NUM, HIDDEN_DIM, nb_epoch))



Epoch: 0



/home/aoncevay/.conda/envs/chana/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Epoch 1/1
16443/16443 [==============================] - 93s 6ms/step - loss: 2.4602
1 the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the store and the stor

Epoch: 1

Epoch 1/1
16443/16443 [==============================] - 94s 6ms/step - loss: 1.8765
the stood and the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the 

**PRUEBA errónea de un checkpoint anterior**
<br>Al reiniciar el notebook, y cargar un checkpoint sin cargar correctamente el diccionario:

In [16]:
WEIGHTS = "checkpoint_layer_2_hidden_250_epoch_10_run1.hdf5"
# Loading the trained weights
model.load_weights(WEIGHTS)
generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, ix_to_char)
print('\n\n')

#El ejemplo impreso en realidad es usando un "diccionario" re-ejecutado y no cargado desde el archivo
#Por eso hay inconsistencias en los caracteres (el mapeo se ha desordenado)

’.hO1®.dO)®OxnD.’i.6O8w.)Y . Y.hww. 6w.h Y)wh.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w.h HwO7.Yn. 6w




**Segundo intento de ENTRENAMIENTO**

In [ ]:
nb_epoch = 0
while True:
    print('\n\nEpoch: {}\n'.format(nb_epoch))
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, epochs=1)
    nb_epoch += 1
    generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, ix_to_char)
    if nb_epoch % 10 == 0:
        model.save_weights('checkpoint_layer_{}_hidden_{}_epoch_{}.hdf5'.format(LAYER_NUM, HIDDEN_DIM, nb_epoch))



Epoch: 0

Epoch 1/1
16443/16443 [==============================] - 48s 3ms/step - loss: 2.7454
un the he the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the th

Epoch: 1

Epoch 1/1
16443/16443 [==============================] - 47s 3ms/step - loss: 2.1540
/ the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the store the 

**PRUEBA correcta de un checkpoint anterior**
<br>Si instancian el modelo y sus parametros (ejecutando algunas celdas preliminares), y tienen los 2 archivos requeridos (.pickle y .hdf5) pueden generar el texto. 
<br>En el ejemplo de LOTR: `VOCAB_SIZE = 84` (si desean probarlo, se adjuntar los pesos y el diccionario, pero no el texto)

In [12]:
#Cuidar de no reemplazar el pickle original
with open('ix_to_char.pickle', 'rb') as handle:
    ix_to_char = pickle.load(handle)
    
WEIGHTS = "checkpoint_layer_2_hidden_250_epoch_60.hdf5"
# Loading the trained weights
model.load_weights(WEIGHTS)
generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, ix_to_char)
print('\n\n')

could see that a wind was chilled and leaned from his head and earth. Twitching the horse shifted in a could began to spread from him or was to see the wall and staring and half-train. It became green spider. There was no sign on the ground and westward.

'Why so, what he was good! ' said Sam. 'So master to see you say thar,' said Aragorn.

'Come back!' said Treebeard. 'But I have no with the fountain of the Marks on the fire to watce the deed of your own friends chooney things to think that you


